# LLM Prototyping Notebook

Quick experiments with LLM providers

In [31]:
# Load environment and imports
import os
from dotenv import load_dotenv
load_dotenv()

import sys
sys.path.append('.')

from modules.llm_provider import agent
from pydantic import BaseModel, Field
from typing import Literal, List, Optional, Union
from enum import Enum
import json
from time import time
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)

# ⚠️ Clear All Outputs before committing to prevent leaking sensitive data! ⚠️

# Changelog

All notable changes to **Screening Cheatsheet Response Schema** will be documented in this cell.

The format is based on [Keep a Changelog](https://keepachangelog.com/en/1.1.0/),
and this project adheres to [Semantic Versioning](https://semver.org/spec/v2.0.0.html).

## [Unreleased]
### Added
- Example cheat sheets templates, inputs, and outputs
### Changed
- Latest changelog revisions
- Updated `.env.example` to follow latest changes
- Response options are dynamically detected from input
## [0.0.1-alpha.2+8404e0a] - 2025-06-17
### Added
- `FinalDecision`
- Default flow diagram pseudocode in `ScreeningCheatsheet`
- `is_last_step` in `FlowDiagramStep` for pseudocode to make sense
- Some [provisions](https://github.com/pvzhelnov/cheatsheet_parser/commit/7c6fae296de066c761db7a4dd7a224eebf64ae7f) against leaking sensitive data
- This changelog
### Changed
- Definitions in `FlowDiagramStep` to handle final decision
- Handling of flow diagram in `ScreeningCheatsheet` to accommodate additions
- Migrated the repo to a [more recent version](https://github.com/pvzhelnov/gerpa/tree/1ff0f1bcd1c63e4dd27a3f0b4e052f0bbad70bb6) of GERPA
- Quite importantly, the migration to new GERPA led to some changes to default LLM inference settings, in particular, all agents are now run with the following config (unless requested otherwise):
  - `temperature` = 0.0
  - `top_k` = 40
  - `top_p` = 0.95
  - `seed` = 42 (if offered by provider)
  - `safety_settings` set to provider-specific values to turn all safety provisions off
### Tested
- On the same sample cheat sheet
## 0.0.1-alpha.1+outside.of.this.repo - 2025-06-03
### Added
- `ScreeningCheatsheet` and all the scaffolding
- System instruction (SHA-256 hash: `64362907`)
- Model used: `gemini-2.5-flash-preview-05-20`
### Tested
- On a sample cheat sheet

[unreleased]: https://github.com/pvzhelnov/cheatsheet_parser/compare/8404e0a0c6332addeb2992cfda5d1ed3f68a2469...HEAD
[0.0.1-alpha.2+8404e0a]: https://github.com/pvzhelnov/cheatsheet_parser/tree/8404e0a0c6332addeb2992cfda5d1ed3f68a2469

In [32]:
class CheatsheetQuestionUID(BaseModel):
    unique_question_id: int

# TO DO: Separate into two LLM calls and implement using generics

class ResponseOptionUniqueLiteral(BaseModel):
    string_value: str = Field(..., description="Capture the literal value verbatim as described in the input file(s).")
    unique_literal: Literal['Yes', 'No', 'Maybe', 'Other'] = Field(..., description="May occasionally include other literal value types depending on context. Select the closest one in case of obvious variants, or select Other if no matching literal value is found.")

class ResponseOption(BaseModel):
    value: ResponseOptionUniqueLiteral
    notes: List[str] = Field(..., description="if the study...")

class CheatsheetQuestion(BaseModel):
    question_uid: CheatsheetQuestionUID
    question_formulation: str
    responses: List[ResponseOption]
    question_note: Optional[str] = Field(..., description="Any question-wide note(s), if present.")

class FinalDecision(BaseModel):
    decision: Literal['Exclude', 'Include']

class OnResponseOption(BaseModel):
    response_option_unique_literal: ResponseOptionUniqueLiteral
    action_to_take: Union[CheatsheetQuestionUID, FinalDecision]

class FlowDiagramStep(BaseModel):
    step_id: int
    is_last_step: bool
    on_response_option: List[OnResponseOption]

class ScreeningCheatsheet(BaseModel):
    allowed_response_options: List[ResponseOptionUniqueLiteral] = Field(..., description="Deduce these from the entire preceding context. Consider obviously equivalent response options like case differences or typos to be equivalent and select the most consistent variant.")
    questions: List[CheatsheetQuestion]
    flow_diagram_detected: bool = Field(..., description="Is there a flow diagram in the input file(s)?")
    flow_diagram: List[FlowDiagramStep] = Field(..., description="If flow_diagram_detected, extract it. Otherwise implement the following pseudocode: { While is_last_step is False: { If Maybe/Yes Then Proceed to next step_id, Else Exclude } ; Once is_last_step is True: { If Maybe/Yes Then Include, Else Exclude } }")

print(json.dumps(ScreeningCheatsheet.model_json_schema(), indent=2, ensure_ascii=False))

{
  "$defs": {
    "CheatsheetQuestion": {
      "properties": {
        "question_uid": {
          "$ref": "#/$defs/CheatsheetQuestionUID"
        },
        "question_formulation": {
          "title": "Question Formulation",
          "type": "string"
        },
        "responses": {
          "items": {
            "$ref": "#/$defs/ResponseOption"
          },
          "title": "Responses",
          "type": "array"
        },
        "question_note": {
          "anyOf": [
            {
              "type": "string"
            },
            {
              "type": "null"
            }
          ],
          "description": "Any question-wide note(s), if present.",
          "title": "Question Note"
        }
      },
      "required": [
        "question_uid",
        "question_formulation",
        "responses",
        "question_note"
      ],
      "title": "CheatsheetQuestion",
      "type": "object"
    },
    "CheatsheetQuestionUID": {
      "properties": {
        "uniq

In [33]:
system_instruction = "You extract accurately data from a systematic review screening cheatsheet."

In [34]:
# Create agent
LLMPROVIDER = "gemini"  # or "openrouter", "ollama"
llm_agent = agent(
    LLMPROVIDER,
    ScreeningCheatsheet,
    system_instruction=system_instruction,
    model_name="gemini-2.5-flash-preview-05-20")

# Test prompt
prompt = [
    "Here is the cheatsheet template:",
    f"{os.getenv("CHEATSHEET_PDF_PATH")}"  # note: docx is apparently unsupported
    #"Here is the cheatsheet flow diagram:",
    #f"{os.getenv("CHEATSHEET_FLOW_DIAGRAM_PDF_PATH")}"
]

start_time = time()
response = llm_agent(prompt)
end_time = time()
print(f"Execution time: {end_time - start_time:.4f} seconds")

INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/upload/v1beta/files "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/upload/v1beta/files?upload_id=ABgVH8-UBrWLqkL7HQjNJA8iPHWT8v9ptQ5JKRln1hwd_NlBSTwhbY09-V2fmlmwbKi2HkDB0fGn4gbLDGyrpz9puURDfNKCr9x0OnLVGj9ubHk&upload_protocol=resumable "HTTP/1.1 200 OK"
INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-05-20:generateContent "HTTP/1.1 200 OK"
INFO:google_genai.models:AFC remote call 1 is done.
2025-07-15 13:56:50,140 - llm_agent_llm_provider - INFO - LLM Response: {
  "provider": {
    "provider_name": "gemini",
    "safety_settings": "[SafetySetting(method=None, category=<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 'HARM_CATEGORY_HATE_SPEECH'>, threshold=<HarmBlockThreshold.BLOCK_NONE: 'BLOCK_NONE'>), SafetySetting(method=None, category=<HarmCategory.H

Execution time: 16.4418 seconds


In [35]:
%%script echo skip
# Experiment with different providers
providers = ["gemini", "openrouter", "ollama"]

for provider in providers:
    try:
        print(f"\n--- Testing {provider} ---")
        test_agent = agent(provider)
        response = test_agent("What is the capital of France?")
        print(f"Response: {response.content[:100]}...")
    except Exception as e:
        print(f"Error with {provider}: {e}")

skip


python(99714) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
